In [2]:
def car(env):
    while True:
        print('Start parking at %d' % env.now)
        parking_duration = 5
        yield env.timeout(parking_duration)

        print('Start driving at %d' % env.now)
        trip_duration = 2
        yield env.timeout(trip_duration)

In [3]:
import simpy
env = simpy.Environment()
env.process(car(env))
env.run(until=50)

Start parking at 0
Start driving at 5
Start parking at 7
Start driving at 12
Start parking at 14
Start driving at 19
Start parking at 21
Start driving at 26
Start parking at 28
Start driving at 33
Start parking at 35
Start driving at 40
Start parking at 42
Start driving at 47
Start parking at 49


In [20]:
class Car(object):
    def __init__(self, env):
        self.env = env
        # Start the run process everytime an instance is created.
        self.action = env.process(self.run())

    def run(self):
        while True:
            print('Start parking and charging at %d' % self.env.now)
            charge_duration = 5
            # We yield the process that process() returns
            # to wait for it to finish
            yield self.env.process(self.charge(charge_duration))

            # The charge process has finished and
            # we can start driving again.
            print('Start driving at %d' % self.env.now)
            trip_duration = 2
            yield self.env.timeout(trip_duration)

    def charge(self, duration):
        yield self.env.timeout(duration)
        
    def driver(env, car):
        yield env.timeout(3)
        car.action.interrupt()

In [21]:
env = simpy.Environment()
car = Car(env)
env.run(until=15)

Start parking and charging at 0
Start driving at 5
Start parking and charging at 7
Start driving at 12
Start parking and charging at 14


In [6]:
def driver(env, car):
    yield env.timeout(3)
    car.action.interrupt()

In [7]:
env = simpy.Environment()
car = Car(env)
env.run(until=15)

Start parking and charging at 0
Start driving at 5
Start parking and charging at 7
Start driving at 12
Start parking and charging at 14


In [8]:
class Car(object):
    def __init__(self, env):
        self.env = env
        self.action = env.process(self.run())
    
    def run(self):
        while True:
            print('Start parking and charging at %d' % self.env.now)
            charge_duration = 5
            # We may get interrupted while charging the battery
            try:
                yield self.env.process(self.charge(charge_duration))
            except simpy.Interrupt:
                # When we received an interrupt, we stop charging and
                # switch to the "driving" state
                print('Was interrupted. Hope, the battery is full enough ...')
            print('Start driving at %d' % self.env.now)
            trip_duration = 2
            yield self.env.timeout(trip_duration)
    
    def charge(self, duration):
        yield self.env.timeout(duration)

In [9]:
env = simpy.Environment()
car = Car(env)
env.run(until=15)

Start parking and charging at 0
Start driving at 5
Start parking and charging at 7
Start driving at 12
Start parking and charging at 14


In [10]:
def car(env, name, bcs, driving_time, charge_duration):
    
    # Simulate driving to the BCS
    yield env.timeout(driving_time)
    
    # Request one of its charging spots
    print('%s arriving at %d' % (name, env.now))
    with bcs.request() as req:
        yield req
        
        # Charge the battery
        print('%s starting to charge at %s' % (name, env.now))
        yield env.timeout(charge_duration)
        print('%s leaving the bcs at %s' % (name, env.now))

In [11]:
env = simpy.Environment()
bcs = simpy.Resource(env, capacity=2)

In [12]:
for i in range(4):
    env.process(car(env, 'Car %d' % i, bcs, i*2, 5))

In [13]:
env.run()

Car 0 arriving at 0
Car 0 starting to charge at 0
Car 1 arriving at 2
Car 1 starting to charge at 2
Car 2 arriving at 4
Car 0 leaving the bcs at 5
Car 2 starting to charge at 5
Car 3 arriving at 6
Car 1 leaving the bcs at 7
Car 3 starting to charge at 7
Car 2 leaving the bcs at 10
Car 3 leaving the bcs at 12


In [14]:
class Car(object):
    def __init__(self, env):
        self.env = env
        # Start the run process everytime an instance is created.
        self.action = env.process(self.run())

    def run(self):
        while True:
            print('Start parking and charging at %d' % self.env.now)
            charge_duration = 5
            # We yield the process that process() returns
            # to wait for it to finish
            yield self.env.process(self.charge(charge_duration))

            # The charge process has finished and
            # we can start driving again.
            print('Start driving at %d' % self.env.now)
            trip_duration = 2
            yield self.env.timeout(trip_duration)

    def charge(self, duration):
        yield self.env.timeout(duration)
        
class Car(object):
    def __init__(self, env):
        self.env = env
        self.action = env.process(self.run())
    
    def run(self):
        while True:
            print('Start parking and charging at %d' % self.env.now)
            charge_duration = 5
            # We may get interrupted while charging the battery
            try:
                yield self.env.process(self.charge(charge_duration))
            except simpy.Interrupt:
                # When we received an interrupt, we stop charging and
                # switch to the "driving" state
                print('Was interrupted. Hope, the battery is full enough ...')
            print('Start driving at %d' % self.env.now)
            trip_duration = 2
            yield self.env.timeout(trip_duration)
    
    def charge(self, duration):
        yield self.env.timeout(duration)

def car(env, name, bcs, driving_time, charge_duration):
    
    # Simulate driving to the BCS
    yield env.timeout(driving_time)
    
    # Request one of its charging spots
    print('%s arriving at %d' % (name, env.now))
    with bcs.request() as req:
        yield req
        
        # Charge the battery
        print('%s starting to charge at %s' % (name, env.now))
        yield env.timeout(charge_duration)
        print('%s leaving the bcs at %s' % (name, env.now))        
        
env = simpy.Environment()
car = Car(env)
env.run(until=15)



Start parking and charging at 0
Start driving at 5
Start parking and charging at 7
Start driving at 12
Start parking and charging at 14


In [1]:
# https://simpy.readthedocs.io/en/latest/simpy_intro/basic_concepts.html
import simpy

def clock(env, name, tick):
    while True:
        print name, env.now
        yield env.timeout(tick)

env = simpy.Environment()
env.process(clock(env, 'fast', 0.5))
env.process(clock(env, 'slow', 1))
env.run(until=2)

fast 0
slow 0
fast 0.5
slow 1
fast 1.0
fast 1.5


In [17]:
class Customers(object):
    def __init__(self, env):
        self.env = env
        # Start the run process everytime an instance is created.
        self.action = env.process(self.run())
        
    def run(self):
        while True:
            print('Start serving customer at %d' % self.env.now)
            service_duration = 5
            yield self.env.timeout(service_duration)
            
            print('Start being idle at %d' % self.env.now)
            idle_duration = 2
            yield self.env.timeout(idle_duration)
            
env = simpy.Environment()
customers = Customers(env)
env.run(until=15)

Start serving customer at 0
Start being idle at 5
Start serving customer at 7
Start being idle at 12
Start serving customer at 14


In [19]:
env = simpy.Environment()
car = Car(env)
env.run(until=15)

Start parking and charging at 0


AttributeError: 'Car' object has no attribute 'charge'

In [22]:
import simpy

def main():
    env = simpy.Environment()
    env.process(traffic_light(env))
    env.run(until=300)
    print("Simulation complete")

def traffic_light(env):
    while True:
        print("Light turned GRN at t= "+str(env.now))
        yield env.timeout(30)
        print("Light turned YEL at t= "+str(env.now))
        yield env.timeout(5)
        print("Light turned RED at t= "+str(env.now))
        yield env.timeout(20)
        


if __name__ == '__main__':
    main()

Light turned GRN at t= 0
Light turned YEL at t= 30
Light turned RED at t= 35
Light turned GRN at t= 55
Light turned YEL at t= 85
Light turned RED at t= 90
Light turned GRN at t= 110
Light turned YEL at t= 140
Light turned RED at t= 145
Light turned GRN at t= 165
Light turned YEL at t= 195
Light turned RED at t= 200
Light turned GRN at t= 220
Light turned YEL at t= 250
Light turned RED at t= 255
Light turned GRN at t= 275
Simulation complete
